# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Получение общей информации о файле

In [1]:
# загружаем и читаем файл в формате csv
import pandas as pd
data=pd.read_csv("/datasets/data.csv")

# получаем статистическую информацию о файле
# выводим первые 10 строк
data.info()
print()
display(data.head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Файл *data* для обработки и анализа содержит:
* 21 525 строк
* 12     колонок


Тип данных в колонках (2) => **float64**:
* *days_employed / общий трудовой стаж в днях*
* *total_income / ежемесячный доход*

Тип данных в колонках (5) => **int64**:
* *children / кол-во детей в семье*
* *dob_years / возраст клиента в годах*
* *education_id / идентификатор уровня образования*
* *family_status_id / идентификатор семейного положения*
* *debt / имел ли задолженность по возврату кредитов*

Тип данных в колонках (5) => **object**:
* *education / уровень образования клиента*
* *family_status / семейное положение*
* *gender / пол клиента*
* *income_type / тип занятости*
* *purpose / цель получения кредита*

Предоставленные данные предварительно оцениваются как достаточные для ответов на вопрос:
>влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

На этапе предобработки данных необходимо:

**уделить внимание:**
* пропуски (2 174) в колонках *days_employed* и *total_income*
* тип данных float64 в колонках *days_employed* и *total_income*
* смешение верхнего и нижнего регистра в данных типа *object*

**проверить качество данных в колонках:**
* children / кол-во детей в семье
* family_status / семейное положение
* debt / имел ли задолженность по возврату кредитов
* total_income / ежемесячный доход
* purpose / цель получения кредита

т.к. эти данные будут в фокусе анализа и потребуются для формирования выводов.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [49]:
# определяем наличие и количество пропущенных значений в таблице,
# сохраняем результат в переменную, выводим результ
data_null = data.isna().sum()
print("Количество строк в колонках с пропусками до обработки:")
print(data_null)
print()

# заполняем пропущенные значения в колонке days_employed значением "0"
data["days_employed"] = data["days_employed"].fillna(0)

# проверяем результат замены
print("Количество строк в колонке days_employed с пропусками после обработки:")
print(data.isna().sum())
print()

# вычисляем медианный доход в подгруппах колонки income_type
# выводим таблицу медианных доходов по типам дохода
median_income = data.groupby("income_type")["total_income"].median()
print("Медианный доход в подгруппах типов доходов:")
print(median_income)
print()

# заменяем значение 0 в колонке total_income в зависимости от типа доходов
# проверяем результат замены, выводим значение минимального дохода
for inc_type in median_income.index:
    data.loc[data['income_type'] == inc_type,'total_income'] = data.loc[data['income_type'] == inc_type,'total_income'].fillna(median_income[inc_type])
    
print("Значение минимального дохода после обработки и замены:")
print(data["total_income"].min())
print()
print("Количество строк в колонках с пропусками после обработки:")
print(data.isna().sum())
print()

# проверяем структуру данных колонки children
print("Данные колонки children до обработки:")
children_value = data["children"].value_counts()
print(children_value)
print()

# производим замену -1 на 0,
# проводим проверку замены
data["children"] = data["children"].replace(-1, 0)
children_minus1_count = data.loc[data.loc[:, "children"] == -1]["children"].count()
print("Количество записей со значением -1 детей после обработки:")
print(children_minus1_count)
print()

# определяем среднее значение количества детей в каждой подгруппе семейного статуса,
children_pivot = data.pivot_table(index = ["family_status"], values = "children", aggfunc = "mean")
print("Среднее количество детей в подгруппах семейного статуса:")
print(children_pivot)
print()

# производим замену 20 на 1,
# проводим проверку замены
data["children"] = data["children"].replace(20, 1)
children_20_count = data.loc[data.loc[:, "children"] == 20]["children"].count()
print("Количество записей со значением 20 детей после обработки:")
print(children_20_count)
print()

# проверяем данные колонки children
print("Данные колонки children после обработки:")
print(data["children"].value_counts())
print()

# проверяем качество данных колонки family_status
print("Данные колонки family_status:")
print(data["family_status"].value_counts())
print()

# проверяем качество данных колонки debt
print("Данные колонки debt:")
print(data["debt"].value_counts())
print()

# проверяем качество данных колонки total_income
print("Данные колонки total_income:")
totinc_max = data["total_income"].max()
totinc_min = data["total_income"].min()
print("Максимальный доход: {:.0f}".format(totinc_max))
print("Минимальный доход: {:.0f}".format(totinc_min))
print()


Количество строк в колонках с пропусками до обработки:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество строк в колонке days_employed с пропусками после обработки:
children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Медианный доход в подгруппах типов доходов:
income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163

**Вывод**

В данных было выявлено **2 174** пропущенных значения в колонках:
* *days_employed*
* *total_income*

что составляет **10%** от общего количества записей и не позволяет проигнорировать - удалить строки, содержащие их.

Информация о трудовом стаже напрямую не связана с поставленной задачей. Принято решение произвести замену пропущенных значений на "0" методом fillna.

Информация о ежемесячном доходе необходима для формирования выводов и потребовала дальнейшей обработки. Для заполнения отсутствущих данных с использованием метода groupby были рассчитаны медианные значения для каждой группы по типам доходов. Отсутствющие значения были заменены на медианные с помощью цикла по принципу соответствия типа дохода.

Учитывая вид пропущенных данных можно предположить, что пропуски вызваны человеческим фактором - непредоставлением достаточного пакета документов со стороны клиента и недостаточным вниманием к этому вопросу со стороны сотрудника банка, оформляющего кредит. В качестве рекомендации можно посоветовать настроить эти поля в CRM банка как обязательные с защитой от преднамеренной ошибки.

Также была проведена проверка данных в колонке *children*. Выявлены сомнительные значения с данными *-1* и *20*.
* Значение *-1* было заменено на значение *0* методом *replace*
* Для замены значений *20* с помощью сводной таблицы были рассчитаны средние значения количества детей в группах семейного статуса. Принято решение заменить *20* на *1*. Замена произведена методом *replace*.

Проверка данных в остальных колонках, требующихся для дальнейшего анализа:
* family_status
* debt
* total_income

показала удовлетворительный результат.

### Замена типа данных

In [50]:
# производим замену типа данных float64 в тип int64 в колонках "days_employed" и "total_income"
data["days_employed"] = data["days_employed"].astype("int")
data["total_income"] = data["total_income"].astype("int")

# проверяем результат замены
print("Типы данных после обработки:")
print()
data.info()
print()


Типы данных после обработки:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB



**Вывод**

Тип данных *float* был заменен на тип данных *int* c применением метода *astype("int")* в колонках *days_employed* и *total_income*. Данная замена особенно в отношении данных колонки *total_income* необходима для дальнейшего проведения исследований и анализа с использованием различных методов, часть из которых может быть применима только к данным типа *int*.

В остальных колонках типы данных оценены как соответствующие для анализа.

### Обработка дубликатов

In [51]:
# приводим данные типа object к нижнему регистру,
# выводим первые 15 строк для проверки
data["education"] = data["education"].str.lower()
data["family_status"] = data["family_status"].str.lower()
data["gender"] = data["gender"].str.lower()
data["income_type"] = data["income_type"].str.lower()
data["purpose"] = data["purpose"].str.lower()

print("Изменение регистра:")
display(data.head(15))
print()

Изменение регистра:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем
7,0,-152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование
8,2,-6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи


In [52]:
# проверяем наличие явных дубликатов в данных,
# сохраняем результат в переменную, выводим результ
data_dupl = data.duplicated().sum()
print("Количество явных дубликатов до обработки:")
print(data_dupl)

# удаляем дубликаты с изменением индексации,
# проверяем результат удаления явных дубликатов
data = data.drop_duplicates().reset_index(drop = True)
print("Количество явных дубликатов после обработки:")
print(data.duplicated().sum())
print()
print("Количество строк после удаления дубликатов:")
print(data.shape)
print()



Количество явных дубликатов до обработки:
71
Количество явных дубликатов после обработки:
0

Количество строк после удаления дубликатов:
(21454, 12)



**Вывод**

Для приведения данных типа *object* к единому нижнему регистру был применен метод *str.lower()*.

После приведения данных к единому нижнему регистру использование метода *data.duplicated().sum()* позволило выявить 71 явный дубликат. Явные дубликаты были удалены методом *data.drop_duplicates().reset_index(drop = True)* с изменением индексов.

### Лемматизация

In [53]:
# импортируем pymystem3 и лемматизируем колонку purpose
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
text = " ".join(data['purpose'])
lemmas = m.lemmatize(text)
print("Счетчик лемм:")
print(Counter(lemmas))
print()


Счетчик лемм:
Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})



In [54]:
# формируем список идентификаторов уникальных целей
purpose_id = ["недвижимость_покупка", "недвижимость_строительство", "недвижимость_ремонт", "автомобиль", "образование", "свадьба", "другое"]

# создаем функцию привязки цели и идентификатора
def purpose_id(purpose):
       for element in purpose:
            if "недвиж" in purpose and "покупк" in purpose:
                return "недвижимость_покупка"
            elif "недвиж" in purpose and "приобр" in purpose:
                return "недвижимость_покупка"
            elif "жил" in purpose and "покупк" in purpose in purpose:
                return "недвижимость_покупка"
            elif "жил" in purpose and "приобр" in purpose:
                return "недвижимость_покупка"
            elif "недвиж" in purpose and "строитель" in purpose:
                return "недвижимость_строительство"
            elif "жил" in purpose and "строитель" in purpose:
                return "недвижимость_строительство"
            elif "недвиж" in purpose and "ремон" in purpose:
                return "недвижимость_ремонт"
            elif "жил" in purpose and "ремон" in purpose:
                return "недвижимость_ремонт"
            elif "недвиж" in purpose and "операц" in purpose:
                return "недвижимость_покупка"
            elif "жил" in purpose and "операц" in purpose:
                return "недвижимость_покупка"
            elif "недвиж" in purpose or "жил" in purpose:
                return "недвижимость_покупка"
            elif "автомоб" in purpose:    
                return "автомобиль"
            elif "образован" in purpose:
                return "образование"
            elif "свадьб" in purpose:
                return "свадьба"
            else:
                return "другое"
        
# добавляем в таблицу колонку purpose_id
# проверяем кол-во уникальных значений,
# определяем долю каждой цели
data["purpose_id"] = data["purpose"]. apply(purpose_id)
data_unique_purp_count = data["purpose_id"].value_counts()
data_unique_purp_share = data_unique_purp_count / data_unique_purp_count.sum() * 100

print("Доля уникальных значений по идентификатору цели:")
print(data_unique_purp_share)
print()
print("Общее количество записей в колонке идентификатор цели:")
print(len(data["purpose_id"]))




Доля уникальных значений по идентификатору цели:
недвижимость_покупка          38.808614
автомобиль                    20.070849
образование                   18.705137
свадьба                       10.832479
недвижимость_строительство     8.753612
недвижимость_ремонт            2.829309
Name: purpose_id, dtype: float64

Общее количество записей в колонке идентификатор цели:
21454


**Вывод**

Лемматизация позволила создать список *идентификаторов уникальных целей* с 6 ключевыми группами:
* недвижимость_покупка       (8328) = 39% записей
* автомобиль                 (4308) = 20% записей
* образование                (4014) = 19% записей
* свадьба                    (2335) = 11% записей
* недвижимость_строительство (1879) =  9% записей
* недвижимость_ремонт        ( 607) =  3% записей

Группа *недвижимость*, как самая крупная, была разбита на 3 по принципу "действий" с недвижимостью.

Записи с общей целью *недвижимость* и *жилье* были добавлены в группу *недвижимость_покупка* на основе предположения, что именно покупка как действие в отношении этих целей является наиоболее вероятным и частым случаем при запросе кредита.

### Категоризация данных

In [55]:
# производим классификацию данных по доходам
# выводим максимальное, минимальное значение,
# находим медиану уникальных значений total_income
print("Максимальное значение дохода: {:1.0f}".format(data["total_income"].max()))
print("Минимальное значение дохода: {:1.0f}".format(data["total_income"].min()))
lst_totinc = data["total_income"].unique()
import statistics
lst_totinc_median = statistics.median(lst_totinc)
print("Медиана дохода: {:1.0f}".format(lst_totinc_median))
print()

#определяем границы категорий дохода
print(data["total_income"].describe())
print()

# создаем список идентификаторов дохода
income_id = ["низкий", "средний", "высокий", "экстремальный"]
# создаем функцию привязки дохода и идентификатора
def income_id(total_income):
    if total_income <= data["total_income"].quantile(0.25):
        return "низкий"
    if total_income <= data["total_income"].quantile(0.50):
        return "средний"
    if total_income <= data["total_income"].quantile(0.75):
        return "высокий"
    else:
        return "экстремальный"
        
# добавляем в таблицу колонку income_id
# проверяем кол-во уникальных значений
data["income_id"] = data["total_income"]. apply(income_id)
print("Количество уникальных значений по идентификатору дохода:")
print(data["income_id"].value_counts())
print()
print("Общее количество записей в колонке идентификатор дохода:")
print(len(data["income_id"]))
print()


Максимальное значение дохода: 2265604
Минимальное значение дохода: 20667
Медиана дохода: 145896

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

Количество уникальных значений по идентификатору дохода:
средний          5479
низкий           5364
экстремальный    5364
высокий          5247
Name: income_id, dtype: int64

Общее количество записей в колонке идентификатор дохода:
21454



In [56]:
# производим классификацию данных по кол-ву детей
# создаем список идентификаторов кол-ва детей
children_id = ["ноль", "один", "два_пять"]

# создаем функцию привязки кол-ва детей и идентификатора
def children_id(children):
    if children == 0:
        return "ноль"
    if children == 1:
        return "один"
    else:
        return "больше двух"
    
# добавляем в таблицу колонку children_id
# проверяем кол-во уникальных значений
data["children_id"] = data["children"]. apply(children_id)
print("Количество уникальных значений по идентификатору кол-ва детей:")
print(data["children_id"].value_counts())
print()
print("Общее количество записей в колонке идентификатор кол-ва детей:")
print(len(data["children_id"]))
print()


Количество уникальных значений по идентификатору кол-ва детей:
ноль           14138
один            4884
больше двух     2432
Name: children_id, dtype: int64

Общее количество записей в колонке идентификатор кол-ва детей:
21454



In [57]:
# группируем значения по запрошенным параметрам,
# добавляем колонку с расчетным параметром % кредитов, возвращенных в срок
children_grouping = data.groupby("children_id")["debt"].agg(["sum", "count"])
children_grouping["debt_intime"] = (children_grouping["count"] - children_grouping["sum"]) /  children_grouping["count"] * 100

famstat_grouping = data.groupby("family_status")["debt"].agg(["sum", "count"])
famstat_grouping["debt_intime"] = (famstat_grouping["count"] - famstat_grouping["sum"]) /  famstat_grouping["count"] * 100

incomeid_grouping = data.groupby("income_id")["debt"].agg(["sum", "count"])
incomeid_grouping["debt_intime"] = (incomeid_grouping["count"] - incomeid_grouping["sum"]) /  incomeid_grouping["count"] * 100

purposeid_grouping = data.groupby("purpose_id")["debt"].agg(["sum", "count"])
purposeid_grouping["debt_intime"] = (purposeid_grouping["count"] - purposeid_grouping["sum"]) /  purposeid_grouping["count"] * 100

# создаем цикл объединения таблиц
tables = children_grouping, famstat_grouping, incomeid_grouping, purposeid_grouping
for table in tables:
    table_1 = children_grouping
    table_2 = famstat_grouping
    table_3 = incomeid_grouping
    table_4 = purposeid_grouping
    joined_table = table_1.append(table_2).append(table_3).append(table_4)
    
joined_table = joined_table.reset_index()

# переименовываем колонки,
# округляем значения в колонке % кредитов в срок до 2х знаков после запятой
joined_table.columns = ["подгруппа", "кол-во просроченных кредитов", "кол-во кредитов всего", "% кредитов в срок"]
joined_table["% кредитов в срок"] = joined_table["% кредитов в срок"].round(2)

# создаем список идентификаторов групп
# создаем привязку группы и идентификатора
# добавляем колонку group_id в таблицу
group_id = ["кол-во детей", "семейное положение", "уровень дохода", "цель кредита"]

children_lst = data["children_id"].unique().tolist()
famstat_lst = data["family_status"].unique().tolist()
incid_lst = data["income_id"].unique().tolist()
purpid_lst = data["purpose_id"].unique().tolist()
def groupid_add (row):
    if row in children_lst:
        return "кол-во детей"
    if row in famstat_lst:
        return "семейное положение"
    if row in incid_lst:
        return "уровень дохода"
    if row in purpid_lst:
        return "цель кредита"
    
joined_table["группа"] = joined_table["подгруппа"].apply(groupid_add)
joined_table = joined_table[["группа", "подгруппа", "кол-во просроченных кредитов", "кол-во кредитов всего", "% кредитов в срок"]]
display(joined_table)

,группа,подгруппа,кол-во просроченных кредитов,кол-во кредитов всего,% кредитов в срок
0,кол-во детей,больше двух,225,2432,90.75
1,кол-во детей,ноль,1064,14138,92.47
2,кол-во детей,один,452,4884,90.75
3,семейное положение,в разводе,85,1195,92.89
4,семейное положение,вдовец / вдова,63,959,93.43
5,семейное положение,гражданский брак,388,4151,90.65
6,семейное положение,женат / замужем,931,12339,92.45
7,семейное положение,не женат / не замужем,274,2810,90.25
8,уровень дохода,высокий,448,5247,91.46
9,уровень дохода,низкий,427,5364,92.04


**Вывод**

Для формирования ответов на вопросы и финальных выводов:

1. Была проведена дополнительная классификация категории *уровень дохода* с идентификаторами *income_id*:

* "низкий" (до 110 000)
* "средний" (100 000 -  150 000)
* "высокий" (150 000 - 200 000)
* "экстремальный" (свыше 200 000)

С помощью методов *max, min, median, describe* были получены данные для определения порогов уровней. С помощью функции данные по доходам были привязаны к идентификаторам.

2. Была проведена дополнительная классификация категории *количество детей* с идентификаторами *children_id*:

* "ноль" (кол-во детей == 0)
* "один" (кол-во детей == 1)
* "больше двух" (кол-во детей 2 и более)

С помощью функции данные по количеству детей были привязаны к идентификаторам.

2. Были выделены 4 основных категории данных:

* "кол-во детей"
* "семейное положение"
* "уровень дохода"
* "цель кредита"

В процессе категоризации был использован метод *groupby*, создан цикл объединения таблиц, рассчитан параметр *% кредитов в срок* как отношение в срок возвращенных кредитов к общему количеству взятых кредитов в подгруппах.

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [58]:
# делаем срез по группе кол-во детей,
# сортируем данные по колонке % кредитов в срок
final_children = joined_table[joined_table["группа"] == "кол-во детей"].sort_values(by = "% кредитов в срок", ascending = False)
display(final_children)
print()


,группа,подгруппа,кол-во просроченных кредитов,кол-во кредитов всего,% кредитов в срок
1,кол-во детей,ноль,1064,14138,92.47
0,кол-во детей,больше двух,225,2432,90.75
2,кол-во детей,один,452,4884,90.75


In [59]:
final_children = joined_table[joined_table["группа"] == "кол-во детей"].sort_values(by = "% кредитов в срок", ascending = False)
display(final_children)

,группа,подгруппа,кол-во просроченных кредитов,кол-во кредитов всего,% кредитов в срок
1,кол-во детей,ноль,1064,14138,92.47
0,кол-во детей,больше двух,225,2432,90.75
2,кол-во детей,один,452,4884,90.75


**Вывод**

Для представления результата был сделан срез данных по группе *кол-во детей*.
Результ показал:

* Наиболее **надежными** плательщиками по кредитам являются представители подгруппы с количеством детей *0*. При этом на них приходится самая большая доля выданных кредитов (66%).

* Представители подгрупп *с детьми* возвращают кредиты **хуже**, что может быть связано с тем, что наличие детей предполагает регулярное возникное непредвиденных расходов. Параметр возврата кредита в срок у них является одинаковым (90,75%) но при этом представители подгруппы с *одним* ребенком берут кредиты в два раза чаще (доля 23%), чем представители подгруппы *больше двух* (доля 11%), что может быть связано с нежеланием брать на себя кредитные обязательства в ситуации, когда количество детей предпологает, что большая часть бюджета уходит на их содержание.

Общий вывод - **зависимость между наличием детей и возвратом кредита в срок существует**.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [66]:
# делаем срез по группе семейное положение,
# сортируем данные по колонке % кредитов в срок
final_famstat = joined_table[joined_table["группа"] == "семейное положение"].sort_values(by = "% кредитов в срок", ascending = False)
display(final_famstat)
print()

# производим группировку данных по группам семейное положение и кол-во детей
# отфильтровываем и выводим значения с общим количеством полученных кредитов больше 1000
final_famstat_child = data.groupby(["family_status", "children_id"])["debt"].agg(["sum", "count"])
final_famstat_child["debt_intime"] = ((final_famstat_child["count"] - final_famstat_child["sum"]) /  final_famstat_child["count"] * 100).round(2)
final_famstat_child = final_famstat_child.reset_index()
final_famstat_child.columns = ["семейное положение", "кол-во детей", "кол-во просроченных кредитов", "кол-во кредитов всего", "% кредитов в срок"]
display(final_famstat_child[ final_famstat_child["кол-во кредитов всего"] > 1000])

,группа,подгруппа,кол-во просроченных кредитов,кол-во кредитов всего,% кредитов в срок
4,семейное положение,вдовец / вдова,63,959,93.43
3,семейное положение,в разводе,85,1195,92.89
6,семейное положение,женат / замужем,931,12339,92.45
5,семейное положение,гражданский брак,388,4151,90.65
7,семейное положение,не женат / не замужем,274,2810,90.25


,семейное положение,кол-во детей,кол-во просроченных кредитов,кол-во кредитов всего,% кредитов в срок
7,гражданский брак,ноль,229,2735,91.63
8,гражданский брак,один,121,1007,87.98
9,женат / замужем,больше двух,165,1818,90.92
10,женат / замужем,ноль,517,7497,93.10
11,женат / замужем,один,249,3024,91.77
13,не женат / не замужем,ноль,210,2267,90.74


**Вывод**

Для представления результата был сделан срез данных по группе *семейное положение*.

Результ показал:

* Самыми **надежными** плательщиками по кредитам являются *вдовцы и вдовы*. При этом у них самая низкая доля в выданных кредитах (4,5%).

* На **втором** месте идут *разведенные*. При этом их доля в выданных кредитах также невилика (5,6%).

* Самая представительная группа - *семейные* (женат / замужем). При этом процент возврата кредита в срок у них выше среднего по группе. Можно предположить, что в рамках устойчивых семейных отношений погашение кредита планируется и осуществляется исходя из совместного дохода, что позволяет в 92,46% случаев возвращать кредит в срок.

* **Менее надежными** плательщиками являются представители группы, находящиеся в *гражданском браке*. При этом их доля в выданных кредитах составляет 19,4%.

* **Хуже всего** возвращают кредиты представители подгруппы *не женат / не замужем*. Их доля ничтожно мала. Но при скоринге этот аспект следует учитывать.

Для выявления зависимости *семейного положения и количества детей* на факт погашения кредита в срок был применен метод *groupby* и с целью удаления "шумов" произведена фильтрация по значениям общего количества выданных кредитов (> 1 000).

Результ показал:

* В подгруппе *гражданский брак* у клиентов *без детей* % погашения кредитов в срок существенно выше, чем у клиентов с 1 ребенком.

* В подгруппе *женат / замужем* % погашения кредитов в срок снижается с ростом количества детей.

Общий вывод - **зависимость между семейным положением и возвратом кредита в срок существует. Также существует зависисмость между семейным положением, количеством детей и возвратом кредита в срок**.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [67]:
# делаем срез по группе уровень дохода,
# сортируем данные по колонке % кредитов в срок
final_incid = joined_table[joined_table["группа"] == "уровень дохода"].sort_values(by = "% кредитов в срок", ascending = False)
display(final_incid)


,группа,подгруппа,кол-во просроченных кредитов,кол-во кредитов всего,% кредитов в срок
11,уровень дохода,экстремальный,383,5364,92.86
9,уровень дохода,низкий,427,5364,92.04
8,уровень дохода,высокий,448,5247,91.46
10,уровень дохода,средний,483,5479,91.18


**Вывод**

Для представления результата был сделан срез данных по группе *уровень дохода*.

Результ показал:

Распределение представителей группы по подгруппам получилось приблизительно равномерным. При этом Параметр *% возврата кредита в срок* между представителями подгрупп *низкий, высокий, средний* идет с минимальным шагом 0,22 / 0,44 %. Самыми **надежными** плательщиками можно считать представителей подгруппы *экстремальный*. Но в общем можно сделать вывод, что **зависимость между уровнем дохода и возвратом кредита в срок отсутствует**. Скорее всего это объясняется тем, что уровень дохода больше влияет на сумму запрашиваемого кредита, нежели на дисциплину его погашения.

- Как разные цели кредита влияют на его возврат в срок?

In [68]:
# делаем срез по группе цель кредита,
# сортируем данные по колонке % кредитов в срок
final_purpid = joined_table[joined_table["группа"] == "цель кредита"].sort_values(by = "% кредитов в срок", ascending = False)
display(final_purpid)

,группа,подгруппа,кол-во просроченных кредитов,кол-во кредитов всего,% кредитов в срок
14,цель кредита,недвижимость_ремонт,35,607,94.23
13,цель кредита,недвижимость_покупка,603,8326,92.76
15,цель кредита,недвижимость_строительство,144,1878,92.33
17,цель кредита,свадьба,186,2324,92.00
16,цель кредита,образование,370,4013,90.78
12,цель кредита,автомобиль,403,4306,90.64


**Вывод**

Для представления результата был сделан срез данных по группе *цель кредита*.

Результ показал:

* Самыми **надежными** плательщиками по кредитам являются клиенты, запрашивающие кредит на *ремонт недвижимости*. При этом у них самая низкая доля в выданных кредитах (3%).

* На **втором и третьем** месте идут *покупка и строительство недвижимости*. Можно предположить, что сами условия кредитования по этим целям, когда недвижимость чаще всего уходит в залог банку до полной выплаты кредита, мотивируют заемщиков к своевременным выплатам.

* Кредиты, запрошенные на *свадьбу*, также находятся в *средней группе надежности*. Шаг между четвертым и третьим местом по % возврата кредитов в срок составляет 0,31%. Можно сделать предположение, что *свадебные кредиты* не велики по запрашиваемой сумме (по сравнению с кредитами на недвижимость, например), их оплата планируется и производится уже из совместного бюджета супругов, и психологически, вступая в новую совместную жизнь, молодые супруги осознают, что хорошая кредитная история важна для них, т.к. в дальнейшем им скорее всего придется обращаться за кредитом на недвижимость.

* **Хуже всего** возвращают кредиты на *образование и автомобиль*. В случае *образования* можно предположить, что % несвоевременных выплат обусловлен нестабильным доходом кредитующихся. В случае *автомобиля* современный рынок автокредивания характеризуется тем, что кредиты берутся в большинстве случаев не для личного пользования автомобилем, а для работы с агрегаторами такси. В такой модели кредитуемому выгоднее не выплачивать кредит вовремя, а после определенного периода коммерческой эксплуатации возвращать автомобиль банку в счет погашения долга. 

Общий вывод - зависимость между целью кредита и возвратом кредита в срок существует.

## Шаг 4. Общий вывод

В результате обработки и анализа предоставленных данных было выявлено, что семейное положение и количество детей клиента влияют на факт погашения кредита в срок:
>Клиенты в гражданском браке без детей гасят кредиты в срок существенно чаще, чем клиенты с 1 ребенком.

>У клиентов, состоящих в браке, % погашения кредитов в срок снижается с ростом количества детей.

По прямым зависимостям было выявлено:

>**КОЛ-ВО ДЕТЕЙ** наиболее **надежными** плательщиками по кредитам являются клиенты без детей. С ростом количества детей снижается количество кредитов, погашенных в срок.

>**СЕМЕЙНОЕ ПОЛОЖЕНИЕ** самая представительная группа клиентов находится в браке, при этом процент возврата кредита в срок у них выше среднего по группе "семейное положение". Самыми **надежными** плательщиками по кредитам являются овдовевшие, при этом у них самая низкая доля в выданных кредитах. Отделу маркектинга рекомендуется обратить на это внимание. **Хуже всего** возвращают кредиты не состоящие в браке. Рекомендуется учесть этот аспект при построении модели скоринга.

>**ЦЕЛЬ КРЕДИТА** самыми **надежными** плательщиками являются клиенты, получившие кредит на недвижимсоть. **Хуже всего** гасят кредиты на образование и автомобиль. Рекомендуется провести анализ продукта автомобильного кредитования с точки зрения областей его применения и достаточности мотивации к погашению кредита в срок.

>**УРОВЕНЬ ДОХОДА** явной зависимости между уровнем дохода и возвратом кредита в срок не выявлено. Сделано предположение, что уровень дохода больше влияет на сумму запрашиваемого кредита, нежели на дисциплину его погашения.

Также в процессе обработки входных данных от банка было выявлено, что существуют пропуски данных о рабочем стаже и уровне доходов клиента. Рекомендуется проверить настройки CRM и возможно сделать эти поля внесения информации обязательными для заполнения с проверкой / защитой от очевидных ошибок.

Все материалы и расчеты представлены в работе и могут быть использованы при построении модели кредитного скоринга.

